In [58]:
import os
import zipfile
import javalang

dataset_name = r"gson"
BASE_DIR = r"C:\Users\tianfy\Desktop\毕设\数据集\A3"
source_folder = r"D:\IDEA_Projects\TestJavaCode\src\main"
JDK_SOURCE = r"C:\Users\tianfy\.jdks\corretto-1.8.0_382\src.zip"
EXPORT_DIR = fr"C:\Users\tianfy\Desktop\毕设\数据集\A3\{dataset_name}"
PROJECT_DIR = r"D:/IDEA_Projects/TestJavaCode"
MAVEN_COMPILE_NAME = r"compile.txt"
if not os.path.exists(EXPORT_DIR):
    os.makedirs(EXPORT_DIR)
FILE_PATH = fr"{BASE_DIR}\{dataset_name}.txt"
with open(FILE_PATH, "r", encoding="utf-8") as f:
    data = f.read()


In [62]:
testcases = data.split("@Test")
print(f"raw: {len(testcases)}")
testcases = [f"@Test\n{x}" for x in testcases if x.strip() != ""]
print(f"filtered: {len(testcases)}")

raw: 126
filtered: 125


In [63]:
import javalang

count = 0


def is_syntax_error(testcase):
    global count
    try:
        template = fr"""
        public class Generated_{count}_Test{{
            {testcase}
        }}
        """
        tree = javalang.parse.parse(template)
        count += 1
        return False
    except:
        return True

_ = len(testcases)
testcases = [x for x in testcases if not is_syntax_error(x)]
print(f"no syntex error: {len(testcases)} (-{_-len(testcases)})")


no syntex error: 58 (-67)


In [64]:
class_map = {}
import os


def update_class_ref(source_folder, class_map):
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith(".java"):
                class_name = file[:-5]
                root = root.replace("\\", ".")
                root = root.replace("/", ".")
                if "src.main.java." not in root:
                    continue
                ref = root.split("src.main.java.")[1]
                if class_name not in class_map:
                    class_map[class_name] = [ref]
                else:
                    class_map[class_name].append(ref)


def update_class_ref_zip(source_zip, class_map):
    with zipfile.ZipFile(source_zip, 'r') as z:
        for filename in z.namelist():
            if filename.endswith(".java"):
                class_name = filename.split("/")[-1][:-5]
                root = os.path.dirname(filename).replace("/", ".")
                if "java.lang." in root:
                    continue
                if class_name not in class_map:
                    class_map[class_name] = [root]
                else:
                    class_map[class_name].append(root)


update_class_ref(source_folder, class_map)
update_class_ref_zip(r"C:\Users\tianfy\.m2\repository\junit\junit\4.13.2\junit-4.13.2-sources.jar", class_map)
update_class_ref_zip(JDK_SOURCE, class_map)


def class_name_2_import(namelist: list[str]):
    imports = [
        "import org.junit.Test;",
        "import static org.junit.Assert.*;",
        "import static org.hamcrest.CoreMatchers.*;",

    ]
    for class_name in namelist:
        imports.append(f"import {class_map[class_name][0]}.{class_name};")
    imports = list(set(imports))
    return "\n".join(imports)

In [65]:
def get_testcase_imports(testcase):
    tokens = list(javalang.tokenizer.tokenize(testcase))
    identifier_tokens = [x.value for x in tokens if isinstance(x, javalang.tokenizer.Identifier)]
    valid_identifier_tokens = [x for x in identifier_tokens if x in class_map]
    return class_name_2_import(valid_identifier_tokens)


for index, testcase in enumerate(testcases):
    with open(fr"{EXPORT_DIR}\Generated_{index}_Test.java", "w", encoding="utf-8") as f:
        template = f"""

        {get_testcase_imports(testcase)}
        public class Generated_{index}_Test{{
            {testcase}
        }}
        """
        f.write(template.strip())
_ = len(testcases)

In [66]:
compile_txt = open(fr"{PROJECT_DIR}/{MAVEN_COMPILE_NAME}", "r", encoding="utf16").readlines()
compile_txt = [x.strip() for x in compile_txt if x.startswith("[ERROR]") and PROJECT_DIR in x]
compile_txt = "\n".join(compile_txt)
pattern = f"{PROJECT_DIR}/src/test/java/(.*?):\["
import re
# search
matches = re.findall(pattern, compile_txt)
# remove duplicates
matches = list(set(matches))
# remove file
for match in matches:
    os.remove(fr"{PROJECT_DIR}/src/test/java/{match}")

In [67]:
# 统计剩余的测试用例个数
import os
count = 0
for root, dirs, files in os.walk(fr"{PROJECT_DIR}/src/test/java"):
    for file in files:
        if file.startswith("Generated_") and file.endswith(".java"):
            count += 1
print(count, len(testcases) - count)

0 58
